In [ ]:
"""
======================================================================
飲料罐底部圓形偵測與日期影像擷取系統 (改進版 v3)
Bottle Bottom Circle Detection System with Timeout Protection
======================================================================
作者：Fly Eddie
日期：2025年11月
版本：3.0 (含超時保護、異常處理、資源管理)
專案：OpticalInspect - 工業級光學檢測系統

改進特性（v3.0）：
1. 超時保護機制 - 防止程式無限期卡住
2. 完善的異常處理 - 捕捉所有可能的錯誤
3. 資源自動釋放 - try-finally 確保正確清理
4. Kernel 響應保護 - 支援隨時中斷
5. 執行緒型超時 - Windows 系統相容
======================================================================
"""

# ======================================================================
# 第一部分：導入必要的模組和套件
# ======================================================================

import cv2
import numpy as np
from pathlib import Path
import time
from datetime import datetime
import threading

# ======================================================================
# 第二部分：全域常數定義與系統參數設定
# ======================================================================

# 攝影機參數
CAMERA_INDEX = 0
CAMERA_FRAME_WIDTH = 1280
CAMERA_FRAME_HEIGHT = 720
CAMERA_FPS = 30
CAMERA_INIT_TIMEOUT = 3  # 攝影機初始化超時（秒）
FRAME_READ_TIMEOUT = 2   # 讀幀超時（秒）

# 圓形偵測參數
CANNY_UPPER_THRESHOLD = 150
CANNY_LOWER_THRESHOLD = 50
HOUGH_CIRCLES_DP = 1
HOUGH_CIRCLES_MIN_DIST = 100
HOUGH_CIRCLES_PARAM1 = CANNY_UPPER_THRESHOLD
HOUGH_CIRCLES_PARAM2 = 30
HOUGH_CIRCLES_MIN_RADIUS = 50
HOUGH_CIRCLES_MAX_RADIUS = 300

# 影像處理參數
BLUR_KERNEL_SIZE = (15, 15)
CAPTURE_IMAGE_QUALITY = 95
CAPTURE_COOLDOWN_SECONDS = 1.5

# UI 按鈕參數
BUTTON_WIDTH = 300
BUTTON_HEIGHT = 60
BUTTON_X_CENTER = CAMERA_FRAME_WIDTH // 2
BUTTON_Y = CAMERA_FRAME_HEIGHT - 100
BUTTON_LEFT = BUTTON_X_CENTER - BUTTON_WIDTH // 2
BUTTON_RIGHT = BUTTON_X_CENTER + BUTTON_WIDTH // 2
BUTTON_TOP = BUTTON_Y - BUTTON_HEIGHT // 2
BUTTON_BOTTOM = BUTTON_Y + BUTTON_HEIGHT // 2
BUTTON_COLOR = (100, 200, 100)
BUTTON_TEXT_COLOR = (255, 255, 255)

# 輸出目錄與檔案設定
OUTPUT_DIR = Path("outputs/bottle_detection")
CIRCLE_CROPS_DIR = OUTPUT_DIR / "circle_crops"
FULL_FRAMES_DIR = OUTPUT_DIR / "full_frames"
LOG_FILE = OUTPUT_DIR / "detection_log.txt"

# 顯示與 UI 參數
WINDOW_NAME = "Bottle Bottom Circle Detection - Press 'q' or ESC to exit"
CIRCLE_COLOR = (0, 255, 0)
CIRCLE_LINE_THICKNESS = 2
CIRCLE_CENTER_RADIUS = 5
CIRCLE_CENTER_COLOR = (0, 0, 255)

# 按鍵代碼定義
KEY_Q = ord('q')
KEY_ESC = 27
KEY_S = ord('s')

# ======================================================================
# 第三部分：程式狀態類別與超時控制
# ======================================================================

class ProgramState:
    """程式狀態管理類別"""
    def __init__(self):
        self.is_paused = False
        self.paused_frame = None
        self.button_hovered = False

state = ProgramState()

def execute_with_timeout(func, timeout_seconds, *args, **kwargs):
    """
    執行函數並設置超時時間
    
    功能說明：
    在獨立執行緒中執行函數，設置超時機制。
    若函數未在指定時間內完成，將返回 None 和錯誤訊息。
    
    參數：
    - func: callable，要執行的函數
    - timeout_seconds: int，超時時間（秒數）
    - *args：傳遞給函數的位置參數
    - **kwargs：傳遞給函數的關鍵字參數
    
    返回值：
    - (result, error_msg) tuple，成功時 error_msg 為 None
    """
    result = [None]
    exception = [None]
    
    def target():
        try:
            result[0] = func(*args, **kwargs)
        except Exception as e:
            exception[0] = e
    
    thread = threading.Thread(target=target, daemon=True)
    thread.start()
    thread.join(timeout=timeout_seconds)
    
    if thread.is_alive():
        return None, "操作超時"
    
    if exception[0]:
        return None, str(exception[0])
    
    return result[0], None

# ======================================================================
# 第四部分：滑鼠事件處理函數
# ======================================================================

def mouse_callback(event, x, y, flags, param):
    """滑鼠事件回調函數"""
    global state
    
    mouse_over_button = (BUTTON_LEFT <= x <= BUTTON_RIGHT and 
                         BUTTON_TOP <= y <= BUTTON_BOTTOM)
    
    if event == cv2.EVENT_MOUSEMOVE:
        state.button_hovered = mouse_over_button
    elif event == cv2.EVENT_LBUTTONDOWN:
        if mouse_over_button and state.is_paused:
            state.is_paused = False
            print("✓ 繼續影像擷取...")

def draw_button(frame):
    """在影像上繪製交互式 UI 按鈕"""
    global state
    
    button_color = (150, 255, 150) if state.button_hovered else BUTTON_COLOR
    
    cv2.rectangle(frame, (BUTTON_LEFT, BUTTON_TOP), (BUTTON_RIGHT, BUTTON_BOTTOM),
                  button_color, -1)
    cv2.rectangle(frame, (BUTTON_LEFT, BUTTON_TOP), (BUTTON_RIGHT, BUTTON_BOTTOM),
                  (255, 255, 255), 3)
    
    button_text = "Click to Resume"
    font_scale = 0.9
    font_thickness = 2
    (text_width, text_height), baseline = cv2.getTextSize(
        button_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    
    text_x = BUTTON_X_CENTER - text_width // 2
    text_y = BUTTON_Y + text_height // 2
    
    cv2.putText(frame, button_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX,
                font_scale, BUTTON_TEXT_COLOR, font_thickness)
    
    return frame

def draw_pause_overlay(frame):
    """在影像上繪製暫停狀態覆蓋層"""
    cv2.putText(frame, "[PAUSED]", (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (0, 0, 255), 3)
    
    tip_text = "Image captured and paused"
    (text_width, text_height), _ = cv2.getTextSize(
        tip_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
    
    tip_x = (CAMERA_FRAME_WIDTH - text_width) // 2
    tip_y = CAMERA_FRAME_HEIGHT // 2 - 40
    
    cv2.putText(frame, tip_text, (tip_x, tip_y), cv2.FONT_HERSHEY_SIMPLEX,
                0.8, (0, 255, 255), 2)
    
    return frame

# ======================================================================
# 第五部分：系統初始化函數
# ======================================================================

def create_output_directories():
    """建立所有必要的輸出目錄"""
    try:
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        CIRCLE_CROPS_DIR.mkdir(parents=True, exist_ok=True)
        FULL_FRAMES_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✓ 輸出目錄已建立：{OUTPUT_DIR}")
    except Exception as e:
        print(f"✗ 錯誤：無法建立輸出目錄 - {e}")

def initialize_camera():
    """初始化攝影機硬體與設定（含超時保護）"""
    
    print("正在初始化攝影機（超時時間：3秒）...")
    
    def init_func():
        cap = cv2.VideoCapture(CAMERA_INDEX)
        time.sleep(0.5)
        return cap
    
    cap, error = execute_with_timeout(init_func, CAMERA_INIT_TIMEOUT)
    
    if error:
        print(f"✗ 錯誤：攝影機初始化失敗 - {error}")
        return None
    
    if cap is None or not cap.isOpened():
        print(f"✗ 錯誤：無法開啟攝影機（索引 {CAMERA_INDEX}）")
        return None
    
    # 設定攝影機參數
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_FRAME_HEIGHT)
    cap.set(cv2.CAP_PROP_FPS, CAMERA_FPS)
    
    actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    actual_fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"✓ 攝影機已初始化")
    print(f"  解析度：{actual_width} × {actual_height} 像素")
    print(f"  幀率：{actual_fps:.1f} FPS")
    print()
    
    return cap

def display_startup_info():
    """顯示程式啟動訊息與說明"""
    print("=" * 70)
    print("飲料罐底部圓形偵測與日期影像擷取系統 (改進版 v3.0)")
    print("含超時保護、異常處理、資源管理")
    print("=" * 70)
    print()
    print("[系統參數]")
    print(f"  圓形最小半徑：{HOUGH_CIRCLES_MIN_RADIUS} 像素")
    print(f"  圓形最大半徑：{HOUGH_CIRCLES_MAX_RADIUS} 像素")
    print(f"  攝影機初始化超時：{CAMERA_INIT_TIMEOUT} 秒")
    print(f"  幀讀取超時：{FRAME_READ_TIMEOUT} 秒")
    print()
    print("[按鍵控制]")
    print("  'q' 鍵或 ESC：退出程式")
    print("  's' 鍵：手動擷取當前幀")
    print()
    print("[滑鼠控制]")
    print("  點擊 'Click to Resume' 按鈕：恢復影像擷取")
    print()

# ======================================================================
# 第六部分：圓形偵測核心函數
# ======================================================================

def detect_circles(frame):
    """使用 Hough 圓形變換偵測影像中的圓形"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, BLUR_KERNEL_SIZE, 0)
    edges = cv2.Canny(blurred, CANNY_LOWER_THRESHOLD, CANNY_UPPER_THRESHOLD, apertureSize=3)
    
    circles = cv2.HoughCircles(
        edges, cv2.HOUGH_GRADIENT, dp=HOUGH_CIRCLES_DP,
        minDist=HOUGH_CIRCLES_MIN_DIST, param1=HOUGH_CIRCLES_PARAM1,
        param2=HOUGH_CIRCLES_PARAM2, minRadius=HOUGH_CIRCLES_MIN_RADIUS,
        maxRadius=HOUGH_CIRCLES_MAX_RADIUS)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        return circles, gray, edges
    else:
        return None, gray, edges

def draw_circles_on_frame(frame, circles):
    """在影像上繪製偵測到的圓形及其圓心標記"""
    if circles is None:
        return frame
    
    for circle in circles[0]:
        x = int(circle[0])
        y = int(circle[1])
        r = int(circle[2])
        
        cv2.circle(frame, (x, y), r, CIRCLE_COLOR, CIRCLE_LINE_THICKNESS)
        cv2.circle(frame, (x, y), CIRCLE_CENTER_RADIUS, CIRCLE_CENTER_COLOR, -1)
        
        text = f"r={r}px"
        cv2.putText(frame, text, (x + r + 5, y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2)
    
    return frame

def extract_circle_region(frame, circle):
    """從影像中擷取圓形區域"""
    x = int(circle[0])
    y = int(circle[1])
    r = int(circle[2])
    
    left = x - r
    top = y - r
    right = x + r
    bottom = y + r
    
    height, width = frame.shape[:2]
    
    if left < 0 or top < 0 or right > width or bottom > height:
        return None
    
    cropped_region = frame[top:bottom, left:right]
    return cropped_region

def save_circle_region(cropped_region, frame_number, capture_count):
    """保存擷取的圓形區域影像到磁碟"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"circle_region_{timestamp}_frame_{frame_number:06d}.jpg"
    filepath = CIRCLE_CROPS_DIR / filename
    
    success = cv2.imwrite(str(filepath), cropped_region,
                         [cv2.IMWRITE_JPEG_QUALITY, CAPTURE_IMAGE_QUALITY])
    
    if success:
        print(f"✓ 圓形區域已保存 [{capture_count}]：{filename}")
    else:
        print(f"✗ 保存失敗：{filepath}")
    
    return filepath, success

def log_detection_result(frame_number, circles_count, captured_this_frame):
    """記錄偵測結果到日誌檔案"""
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        captured_status = "Yes" if captured_this_frame else "No"
        log_message = f"[{timestamp}] Frame {frame_number:06d} | Circles: {circles_count} | Captured: {captured_status}\n"
        
        with open(LOG_FILE, "a", encoding="utf-8") as f:
            f.write(log_message)
    except Exception as e:
        print(f"✗ 日誌記錄失敗：{e}")

# ======================================================================
# 第七部分：主程式邏輯
# ======================================================================

def main():
    """主程式函數 - 統整所有功能的主迴圈"""
    
    global state
    
    create_output_directories()
    
    # 初始化攝影機（含超時保護）
    cap = initialize_camera()
    if cap is None:
        print("✗ 無法初始化攝影機，程式終止")
        return
    
    display_startup_info()
    
    # 建立 OpenCV 視窗
    try:
        cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
        cv2.setMouseCallback(WINDOW_NAME, mouse_callback)
    except Exception as e:
        print(f"✗ 無法建立視窗：{e}")
        cap.release()
        return
    
    frame_count = 0
    capture_count = 0
    last_capture_time = 0
    
    print("[即時處理] 開始監視攝影機視頻流...")
    print(f"[等待] 將罐子放在攝影機下方，圓形將自動偵測並暫停")
    print()
    
    try:
        while True:
            # ─────────────────────────────────────────────────────────
            # 根據暫停狀態決定是否讀取新幀
            # ─────────────────────────────────────────────────────────
            
            if state.is_paused:
                display_frame = draw_pause_overlay(state.paused_frame.copy())
                display_frame = draw_button(display_frame)
                cv2.imshow(WINDOW_NAME, display_frame)
                
                key = cv2.waitKey(50)
                if key != -1:
                    key = key & 0xFF
                    if key == KEY_Q or key == KEY_ESC:
                        print("\n✓ 使用者按下退出鍵")
                        break
            else:
                # ─────────────────────────────────────────────────────
                # 正常模式：讀取新幀（含超時保護）
                # ─────────────────────────────────────────────────────
                
                def read_frame():
                    ret, frame = cap.read()
                    return ret, frame
                
                read_result, read_error = execute_with_timeout(
                    read_frame, FRAME_READ_TIMEOUT)
                
                if read_error:
                    print(f"✗ 無法讀取視訊幀：{read_error}")
                    break
                
                if read_result is None:
                    continue
                
                ret, frame = read_result
                
                if not ret or frame is None:
                    print("✗ 無法讀取有效的視訊幀")
                    break
                
                frame_count += 1
                
                # 圓形偵測
                circles, gray, edges = detect_circles(frame)
                frame_with_circles = draw_circles_on_frame(frame.copy(), circles)
                
                # 添加幀數和圓形數的文字信息
                info_text = f"Frame: {frame_count} | Circles: {len(circles[0]) if circles is not None else 0}"
                cv2.putText(frame_with_circles, info_text, (10, 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                # 自動擷取圓形區域（帶防重複機制）
                captured_this_frame = False
                
                if circles is not None:
                    current_time = time.time()
                    
                    if current_time - last_capture_time >= CAPTURE_COOLDOWN_SECONDS:
                        for circle in circles[0]:
                            cropped = extract_circle_region(frame, circle)
                            
                            if cropped is not None:
                                capture_count += 1
                                last_capture_time = current_time
                                captured_this_frame = True
                                
                                filepath, success = save_circle_region(
                                    cropped, frame_count, capture_count)
                                
                                state.is_paused = True
                                state.paused_frame = frame_with_circles.copy()
                                print(f"⏸ 程式已暫停，等待使用者操作...")
                                break
                
                # 記錄此幀的偵測結果
                log_detection_result(frame_count,
                                    len(circles[0]) if circles is not None else 0,
                                    captured_this_frame)
                
                # 顯示影像
                cv2.imshow(WINDOW_NAME, frame_with_circles)
                
                # 按鍵輸入處理
                key = cv2.waitKey(1)
                
                if key == -1:
                    continue
                
                key = key & 0xFF
                
                if key == KEY_Q or key == KEY_ESC:
                    print("\n✓ 使用者按下退出鍵")
                    break
                
                elif key == KEY_S:
                    if circles is not None and len(circles[0]) > 0:
                        circle = circles[0][0]
                        cropped = extract_circle_region(frame, circle)
                        if cropped is not None:
                            capture_count += 1
                            filepath, success = save_circle_region(cropped, frame_count, capture_count)
                            state.is_paused = True
                            state.paused_frame = frame_with_circles.copy()
                            print("✓ 手動擷取完成，程式已暫停")
                    else:
                        print("⚠ 未偵測到圓形，無法擷取")
    
    except KeyboardInterrupt:
        print("\n\n✗ 使用者按下中斷鍵，程式正在清理...")
    
    except Exception as e:
        print(f"\n✗ 發生異常錯誤：{e}")
        import traceback
        traceback.print_exc()
    
    finally:
        # ─────────────────────────────────────────────────────────────
        # 清理與結束階段（確保一定會執行）
        # ─────────────────────────────────────────────────────────────
        
        print("\n[清理] 正在釋放資源...")
        
        try:
            if cap is not None:
                cap.release()
                print("✓ 攝影機資源已釋放")
        except Exception as e:
            print(f"⚠ 釋放攝影機時出現錯誤：{e}")
        
        try:
            cv2.destroyAllWindows()
            print("✓ OpenCV 視窗已關閉")
        except Exception as e:
            print(f"⚠ 關閉視窗時出現錯誤：{e}")
        
        # 打印統計信息
        print("\n[統計信息]")
        print(f"  總幀數：{frame_count}")
        print(f"  已擷取圓形區域數：{capture_count}")
        print(f"  輸出目錄：{OUTPUT_DIR.resolve()}")
        print()
        print("=" * 70)
        print("✓ 程式執行完成")
        print("=" * 70)

# ======================================================================
# 第八部分：程式進入點
# ======================================================================

if __name__ == "__main__":
    """程式進入點 - 條件式執行"""
    try:
        main()
    except Exception as e:
        print()
        print("✗ 發生未預期的異常錯誤：")
        print(f"  {e}")
        import traceback
        traceback.print_exc()
